In [1]:
# Standard libraries
import os
import numpy as np
import pandas as pd

# For progress tracking
from tqdm import tqdm

# For plotting (if needed)
import matplotlib.pyplot as plt

# For machine learning models (we'll use PyTorch in this example)
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim


In [2]:
# Define base dataset directory
base_dir = '/mnt/d/Yandex_Disk/covers_detection_contest/dataset/'  # Replace with the actual path to your dataset

# Define specific data directories
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
splits_dir = os.path.join(base_dir, 'splits')


In [3]:
print(os.path.exists(base_dir))

False


In [4]:
# Load train and validation cliques
train_cliques = np.load(os.path.join(splits_dir, 'train_cliques.npy'), allow_pickle=True)
val_cliques = np.load(os.path.join(splits_dir, 'val_cliques.npy'), allow_pickle=True)

# Load test track IDs
test_ids = np.load(os.path.join(splits_dir, 'test_ids.npy'))


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/d/Yandex_Disk/covers_detection_contest/dataset/splits\\train_cliques.npy'

In [5]:
print("Тип train_cliques:", type(train_cliques))
print("Форма train_cliques:", train_cliques.shape)
print("Пример данных train_cliques:", train_cliques[:5])


NameError: name 'train_cliques' is not defined

In [6]:
print('Number of training cliques:', len(train_cliques))
print('Number of validation cliques:', len(val_cliques))
print('Number of test track IDs:', len(test_ids))

# Inspect the first few cliques
print('First training clique:', train_cliques[0])
print('First validation clique:', val_cliques[0])


NameError: name 'train_cliques' is not defined

In [31]:
print(type(train_cliques))  # Check the type of train_cliques
print(train_cliques.shape if isinstance(train_cliques, np.ndarray) else len(train_cliques))  # If it's a numpy array, check the shape; otherwise, get the length
print(train_cliques[0])  # Inspect the first element to understand its structure


<class 'numpy.ndarray'>
(39535,)
1


In [32]:
print('Base directory:', base_dir)
print('Train directory:', train_dir)


Base directory: /mnt/d/Yandex_Disk/covers_detection_contest/dataset/
Train directory: /mnt/d/Yandex_Disk/covers_detection_contest/dataset/train


In [33]:
import os

def find_file_anywhere(track_id, base_dir):
    """
    Search for a file ending with the last three digits of the track ID in the base directory.
    
    Args:
        track_id (int): The track ID to search for.
        base_dir (str): The base directory (train or test).

    Returns:
        str: The full file path if found, otherwise None.
    """
    track_id_str = str(track_id)
    target_suffix = track_id_str[-3:]  # Last three digits of the track ID
    
    # Walk through all subdirectories of the base_dir
    for root, dirs, files in os.walk(base_dir):
        for file_name in files:
            if file_name.endswith(target_suffix + '.npy'):
                print(f"File found: {file_name}")
                return os.path.join(root, file_name)

    print(f"No file found with suffix {target_suffix} in {base_dir}")
    return None

In [34]:
# Find a file in the train directory
sample_track_id = 195038
sample_file_path = find_file_anywhere(sample_track_id, train_dir)

print('Sample file path:', sample_file_path)
print('File exists:', os.path.exists(sample_file_path) if sample_file_path else "No file found")

File found: 100038.npy
Sample file path: /mnt/d/Yandex_Disk/covers_detection_contest/dataset/train/0/3/8/100038.npy
File exists: True


In [35]:
print(type(train_cliques))
print(train_cliques[0])  # Check if it's a list or scalar

<class 'numpy.ndarray'>
1


In [36]:
def load_spectrogram(track_id, data_dir):
    file_path = get_file_path(track_id, data_dir)
    if os.path.exists(file_path):
        spectrogram = np.load(file_path)
        return spectrogram
    else:
        print(f'Spectrogram file not found for track ID {track_id} at {file_path}')
        return None


In [37]:
def get_file_path(track_id, data_dir):
    """
    Search for a file ending with the last three digits of the track ID in the specified directory.
    
    Args:
        track_id (int): The track ID.
        data_dir (str): The base directory (train or test).

    Returns:
        str: The full file path if found, otherwise None.
    """
    track_id_str = str(track_id)
    target_suffix = track_id_str[-3:]  # Last three digits of the track ID
    
    # Walk through all subdirectories of the data_dir
    for root, dirs, files in os.walk(data_dir):
        for file_name in files:
            if file_name.endswith(target_suffix + '.npy'):
                print(f"File found: {file_name} in {root}")
                return os.path.join(root, file_name)

    print(f"No file found in {data_dir} with suffix {target_suffix}")
    return None


In [38]:
sample_spectrogram = load_spectrogram(sample_track_id, train_dir)

if sample_spectrogram is not None:
    print('Spectrogram shape:', sample_spectrogram.shape)  # Print the shape if loaded successfully
else:
    print("Failed to load spectrogram.")


File found: 100038.npy in /mnt/d/Yandex_Disk/covers_detection_contest/dataset/train/0/3/8
Spectrogram shape: (84, 50)


In [1]:
import matplotlib.pyplot as plt

def plot_spectrogram(spectrogram, title='Spectrogram'):
    plt.figure(figsize=(10, 4))
    plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    plt.title(title)
    plt.ylabel('Frequency bins')
    plt.xlabel('Time frames')
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()
    plt.show()

# Plot the sample spectrogram
if sample_spectrogram is not None:
    plot_spectrogram(sample_spectrogram, title=f'Spectrogram for Track ID {sample_track_id}')


ModuleNotFoundError: No module named 'matplotlib'

In [40]:
def normalize_spectrogram(spectrogram):
    # Normalize to zero mean and unit variance
    mean = np.mean(spectrogram)
    std = np.std(spectrogram)
    spectrogram_normalized = (spectrogram - mean) / std
    return spectrogram_normalized


In [41]:
sample_spectrogram_norm = normalize_spectrogram(sample_spectrogram)
print('Normalized spectrogram mean:', np.mean(sample_spectrogram_norm))
print('Normalized spectrogram std:', np.std(sample_spectrogram_norm))


Normalized spectrogram mean: 1.453218e-08
Normalized spectrogram std: 1.0


In [42]:
class SpectrogramDataset(Dataset):
    def __init__(self, track_ids, data_dir, transform=None):
        self.track_ids = track_ids
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.track_ids)
    
    def __getitem__(self, idx):
        track_id = self.track_ids[idx]
        spectrogram = load_spectrogram(track_id, self.data_dir)
        if spectrogram is None:
            # If spectrogram is missing, return zero array or handle accordingly
            spectrogram = np.zeros((10, 100))  # Assuming spectrograms have this shape
        if self.transform:
            spectrogram = self.transform(spectrogram)
        # Convert to tensor
        spectrogram = torch.tensor(spectrogram, dtype=torch.float32)
        return spectrogram, track_id


In [43]:
# Inspect the structure of train_cliques and val_cliques
print(f"Type of train_cliques: {type(train_cliques)}")
print(f"First few elements of train_cliques: {train_cliques[:5]}")


Type of train_cliques: <class 'numpy.ndarray'>
First few elements of train_cliques: [1 2 4 5 6]


In [44]:
train_track_ids = list(set(train_cliques))
val_track_ids = list(set(val_cliques))

print('Total unique training track IDs:', len(train_track_ids))
print('Total unique validation track IDs:', len(val_track_ids))

Total unique training track IDs: 39535
Total unique validation track IDs: 2081


In [45]:
# Assuming each element in train_cliques is a list of track IDs
train_track_ids = [track_id for clique in train_cliques for track_id in clique]
val_track_ids = [track_id for clique in val_cliques for track_id in clique]

# Remove duplicates (if any)
train_track_ids = list(set(train_track_ids))
val_track_ids = list(set(val_track_ids))

print('Total unique training track IDs:', len(train_track_ids))
print('Total unique validation track IDs:', len(val_track_ids))

TypeError: 'numpy.int64' object is not iterable

In [46]:
# Define any transformations (e.g., normalization)
def preprocess_spectrogram(spectrogram):
    # Apply normalization
    spectrogram = normalize_spectrogram(spectrogram)
    # Expand dimensions if needed (e.g., for CNN input channels)
    spectrogram = np.expand_dims(spectrogram, axis=0)  # Shape: (1, freq_bins, time_frames)
    return spectrogram

# Create dataset instances
train_dataset = SpectrogramDataset(train_track_ids, train_dir, transform=preprocess_spectrogram)
val_dataset = SpectrogramDataset(val_track_ids, train_dir, transform=preprocess_spectrogram)

# Create data loaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


In [47]:
class SpectrogramCNN(nn.Module):
    def __init__(self, embedding_dim=128):
        super(SpectrogramCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),  # Input channels = 1
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # Downsample by factor of 2
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # Add more layers if needed
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * (spectrogram_height // 4) * (spectrogram_width // 4), 256),
            nn.ReLU(),
            nn.Linear(256, embedding_dim)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# We need to define spectrogram_height and spectrogram_width based on the data
spectrogram_height = sample_spectrogram_norm.shape[0]
spectrogram_width = sample_spectrogram_norm.shape[1]

print('Spectrogram dimensions:', spectrogram_height, spectrogram_width)


Spectrogram dimensions: 84 50


In [1]:
!python --version


Python 3.10.11


In [1]:
import lzma
import torchvision.transforms as transforms

# Define a transform to resize spectrograms
resize_transform = transforms.Compose([
    transforms.ToTensor(),  # Converts to (C, H, W) format and scales to [0, 1]
    transforms.Resize((128, 256))  # Resize to 128x256
])

# Update the preprocess_spectrogram function
def preprocess_spectrogram(spectrogram):
    spectrogram = normalize_spectrogram(spectrogram)
    spectrogram = np.expand_dims(spectrogram, axis=0)  # Shape: (1, H, W)
    spectrogram = torch.tensor(spectrogram, dtype=torch.float32)
    spectrogram = resize_transform(spectrogram)
    return spectrogram


ModuleNotFoundError: No module named 'torchvision'

In [53]:
import lzma


ModuleNotFoundError: No module named '_lzma'

In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt

print(torch.__version__)


2.5.0+cu124


In [51]:
import sys
print(sys.executable)

/mnt/d/mnt/c/Users/sound/PycharmProjects/venv/bin/python


In [3]:
!pip install numpy pandas matplotlib tqdm torch torchvision scikit-learn



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\sound\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.7 kB ? eta -:--:--
     -------------------------- ----------- 41.0/59.7 kB 393.8 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/59.7 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 59.7/59.7 kB 351.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/167.0 kB ? eta -:--:--
     ------------------------- ------------ 112.6/167.0 kB 6.4 MB/s eta 0:00:01
     -------------------------------------- 167.0/167.0 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
    ---

In [2]:
# Standard libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# For machine learning models
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity

# Define base dataset directory
base_dir = '/mnt/d/Yandex_Disk/covers_detection_contest/dataset/'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
splits_dir = os.path.join(base_dir, 'splits')

# Load train and validation cliques
train_cliques = np.load(os.path.join(splits_dir, 'train_cliques.npy'), allow_pickle=True)
val_cliques = np.load(os.path.join(splits_dir, 'val_cliques.npy'), allow_pickle=True)
test_ids = np.load(os.path.join(splits_dir, 'test_ids.npy'))

# Print dataset information
print('Number of training cliques:', len(train_cliques))
print('Number of validation cliques:', len(val_cliques))
print('Number of test track IDs:', len(test_ids))

# Function to find a file anywhere in base directory
def find_file_anywhere(track_id, base_dir):
    track_id_str = str(track_id)
    target_suffix = track_id_str[-3:]  # Last three digits of the track ID
    for root, dirs, files in os.walk(base_dir):
        for file_name in files:
            if file_name.endswith(target_suffix + '.npy'):
                return os.path.join(root, file_name)
    return None

# Example of finding a file
sample_track_id = 195038
sample_file_path = find_file_anywhere(sample_track_id, train_dir)
print('Sample file path:', sample_file_path)
print('File exists:', os.path.exists(sample_file_path) if sample_file_path else "No file found")

# Function to load a spectrogram
def get_file_path(track_id, data_dir):
    track_id_str = str(track_id)
    target_suffix = track_id_str[-3:]
    for root, dirs, files in os.walk(data_dir):
        for file_name in files:
            if file_name.endswith(target_suffix + '.npy'):
                return os.path.join(root, file_name)
    return None

def load_spectrogram(track_id, data_dir):
    file_path = get_file_path(track_id, data_dir)
    if os.path.exists(file_path):
        return np.load(file_path)
    else:
        return None

sample_spectrogram = load_spectrogram(sample_track_id, train_dir)
if sample_spectrogram is not None:
    print('Spectrogram shape:', sample_spectrogram.shape)
else:
    print("Failed to load spectrogram.")

# Function to plot a spectrogram
def plot_spectrogram(spectrogram, title='Spectrogram'):
    plt.figure(figsize=(10, 4))
    plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    plt.title(title)
    plt.ylabel('Frequency bins')
    plt.xlabel('Time frames')
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()
    plt.show()

if sample_spectrogram is not None:
    plot_spectrogram(sample_spectrogram, title=f'Spectrogram for Track ID {sample_track_id}')

# Normalize a spectrogram
def normalize_spectrogram(spectrogram):
    mean = np.mean(spectrogram)
    std = np.std(spectrogram)
    return (spectrogram - mean) / std

sample_spectrogram_norm = normalize_spectrogram(sample_spectrogram)
print('Normalized spectrogram mean:', np.mean(sample_spectrogram_norm))
print('Normalized spectrogram std:', np.std(sample_spectrogram_norm))

# Define a dataset class
class SpectrogramDataset(Dataset):
    def __init__(self, track_ids, data_dir, transform=None):
        self.track_ids = track_ids
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.track_ids)
    
    def __getitem__(self, idx):
        track_id = self.track_ids[idx]
        spectrogram = load_spectrogram(track_id, self.data_dir)
        if spectrogram is None:
            spectrogram = np.zeros((10, 100))  # Assuming spectrograms have this shape
        if self.transform:
            spectrogram = self.transform(spectrogram)
        spectrogram = torch.tensor(spectrogram, dtype=torch.float32)
        return spectrogram, track_id

# Prepare train and validation data
train_track_ids = [track_id for clique in train_cliques for track_id in clique]
val_track_ids = [track_id for clique in val_cliques for track_id in clique]
train_track_ids = list(set(train_track_ids))
val_track_ids = list(set(val_track_ids))

# Define any transformations
def preprocess_spectrogram(spectrogram):
    spectrogram = normalize_spectrogram(spectrogram)
    spectrogram = np.expand_dims(spectrogram, axis=0)  # Shape: (1, H, W)
    spectrogram = torch.tensor(spectrogram, dtype=torch.float32)
    resize_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((128, 256))
    ])
    spectrogram = resize_transform(spectrogram)
    return spectrogram

train_dataset = SpectrogramDataset(train_track_ids, train_dir, transform=preprocess_spectrogram)
val_dataset = SpectrogramDataset(val_track_ids, train_dir, transform=preprocess_spectrogram)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Define the CNN model
class SpectrogramCNN(nn.Module):
    def __init__(self, embedding_dim=128):
        super(SpectrogramCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * (spectrogram_height // 4) * (spectrogram_width // 4), 256),
            nn.ReLU(),
            nn.Linear(256, embedding_dim)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

spectrogram_height, spectrogram_width = sample_spectrogram_norm.shape
print('Spectrogram dimensions:', spectrogram_height, spectrogram_width)

# Feature extraction
model = SpectrogramCNN(embedding_dim=128)
model.eval()  # Set model to evaluation mode

def extract_features(loader, model):
    features = {}
    with torch.no_grad():
        for spectrograms, track_ids in loader:
            embeddings = model(spectrograms)
            for track_id, embedding in zip(track_ids, embeddings):
                features[track_id.item()] = embedding.numpy()
    return features

# Extract features for test set
test_dataset = SpectrogramDataset(test_ids, test_dir, transform=preprocess_spectrogram)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
test_features = extract_features(test_loader, model)

# Find 100 closest tracks for each test track
results = []
nDCG_scores = []
for query_id, query_embedding in test_features.items():
    distances = []
    for track_id, track_embedding in test_features.items():
        if query_id != track_id:
            distance = cosine_similarity(query_embedding.reshape(1, -1), track_embedding.reshape(1, -1))[0][0]
            distances.append((track_id, distance))
    distances = sorted(distances, key=lambda x: x[1], reverse=True)[:100]
    closest_tracks = [track_id for track_id, _ in distances]
    results.append((query_id, closest_tracks))
    
    # Calculate nDCG@100 for the current query
    dcg = 0.0
    for i, track_id in enumerate(closest_tracks):
        relevance = 1 / np.sqrt(i + 1)
        dcg += relevance
    nDCG_scores.append(dcg)

# Compute average nDCG@100
average_nDCG = np.mean(nDCG_scores)
print(f"Average nDCG@100: {average_nDCG}")


ModuleNotFoundError: No module named 'numpy'